In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle
from tqdm import tqdm
from scipy import stats
import yaml

from graph_generation import create_ER, dataset_nclass_ER, dataset_regression_ER
from models import GCN
from train import Trainer, Dataset
from embedding import Embedding
from experiments import experiment_embedding

import torch
from torch_geometric.loader import DataLoader
device = torch.device('cuda')
import tensorflow as tf

from deepdiff import DeepDiff

In [2]:
config = yaml.safe_load(open("configs.yml"))
dataset_grafi_nx, dataset_labels, list_p= dataset_nclass_ER(config)  # impostare ultimo neuron layer > 1 anche se cambia poi con la grid search
#dataset_grafi_nx, dataset_labels = dataset_regression_ER(config)

Mean connectivity for each node: 12.962 p=0.05
Mean connectivity for each node: 77.661 p=0.3
Labels values: {0, 1}


In [4]:
learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1]

layers1 = [[1, 8, i] for i in range(2, 9, 6)]
layers1_2 = [[1, 16, i] for i in range(2, 17, 2)]

layers1_3 = [[1, 32, i] for i in range(2, 33, 2)]
layers2 = [[1, i, i, 1] for i in range(2, 100, 12)]
layers = layers1 + layers2

num_nodes = [260, 280, 300, 320, 340, 360]

epochs = [0, 1, 3, 10, 50, 5000]

In [35]:
trials = layers1# + layers1_2 + layers1_3
trials = [[1,32,32]]

In [36]:
import copy
def make_config(config, trials):
    configs = []
    #for lr, layer, num_nodes in trials:
    for layer in trials:
        #config['training']['learning_rate'] = lr
        
        config['model']['neurons_per_layer'] = layer
        #config['training']['epochs'] = epoch
        #config['graph_dataset']['Num_nodes'] = num_nodes
        configs.append(copy.deepcopy(config))
    return configs

In [37]:
configs2 = make_config(config, trials)

In [38]:
# carica il master config
with open('configs_21-6-22.pickle', 'rb') as handle:
    configs = pickle.load(handle)

In [39]:
# check su run già fatti
actual_runs = []
for i, c2 in enumerate(configs2):
    cambi = True
    for j, c in enumerate(configs):
        ddiff = DeepDiff(c, c2, ignore_order=False)
        cambi = ddiff.get('values_changed')
        if not cambi:
            print(f'salto i:{i} j:{j}')
            break    
    if cambi:
        actual_runs.append(c2)

In [40]:
print(len(actual_runs), len(configs2))

1 1


In [41]:
#regression: list_p = None,last parameter True
# classificatrion: list_p, last par = False

for i, c in enumerate(actual_runs):
    print(f'Run {i}/{len(actual_runs)}')
    for j in range(2):
        embeddings, trainer, test_loss_list = experiment_embedding(c, dataset_grafi_nx, dataset_labels, list_p, False)
        #corrs, error = embeddings.calc_correlation()
        #c['correlations'] = corrs
        #c['error'] = error 
        c['test_loss'] = test_loss_list
        c['distance_of_means'] = embeddings.distance_of_means   # TODO: sarebbe il caso di mettere una gerarchia di chiavi anche per i risultati
        #c[f'run_num'] = j

Run 0/1
Batch Normalization included
GCN(
  (convs): ModuleList(
    (0): GCNConv(1, 32)
    (1): GCNConv(32, 32)
  )
  (pools): ModuleList()
  (batchnorms): ModuleList(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (leaky): LeakyReLU(negative_slope=0.03)
  (lin): Linear(in_features=32, out_features=2, bias=True)
)
CrossEntropyLoss()
Loading Dataset...


100%|█████████████████████████████████████████| 600/600 [00:29<00:00, 20.10it/s]


Tempo impiegato: 29.939348697662354
runs/lr-0.01_epochs2500_bs100_neurons-1-32-32_21Jun_19-16-45
Run training for 2500 epochs
Epoch: 0	Test loss: 0.0077365722921159535
Epoch: 100	Test loss: 1.0568221695292676e-08
Epoch: 200	Test loss: 3.5928352638671237e-10
Epoch: 300	Test loss: 0.0
Epoch: 400	Test loss: 0.0
Epoch: 500	Test loss: 0.0
Epoch: 600	Test loss: 0.0
Epoch: 700	Test loss: 0.0
Epoch: 800	Test loss: 0.0
Epoch: 900	Test loss: 0.0
Epoch: 1000	Test loss: 0.0
Epoch: 1100	Test loss: 0.0
Epoch: 1200	Test loss: 0.0
Epoch: 1300	Test loss: 0.0
Epoch: 1400	Test loss: 0.0
Epoch: 1500	Test loss: 0.0
Epoch: 1600	Test loss: 0.0
Epoch: 1700	Test loss: 0.0
Epoch: 1800	Test loss: 0.0
Epoch: 1900	Test loss: 0.0
Epoch: 2000	Test loss: 0.0
Epoch: 2100	Test loss: 0.0
Epoch: 2200	Test loss: 0.0
Epoch: 2300	Test loss: 0.0
Epoch: 2400	Test loss: 0.0
Batch Normalization included
GCN(
  (convs): ModuleList(
    (0): GCNConv(1, 32)
    (1): GCNConv(32, 32)
  )
  (pools): ModuleList()
  (batchnorms): Modul

100%|█████████████████████████████████████████| 600/600 [00:33<00:00, 17.78it/s]


Tempo impiegato: 33.82444715499878
runs/lr-0.01_epochs2500_bs100_neurons-1-32-32_21Jun_19-21-28
Run training for 2500 epochs
Epoch: 0	Test loss: 0.007848604851298862
Epoch: 100	Test loss: 1.5563433011480912e-10
Epoch: 200	Test loss: 1.4901160552385464e-11
Epoch: 300	Test loss: 1.4901160552385464e-11
Epoch: 400	Test loss: 2.1523897479151324e-11
Epoch: 500	Test loss: 0.0
Epoch: 600	Test loss: 0.0
Epoch: 700	Test loss: 0.0
Epoch: 800	Test loss: 0.0
Epoch: 900	Test loss: 0.0
Epoch: 1000	Test loss: 0.0
Epoch: 1100	Test loss: 0.0
Epoch: 1200	Test loss: 0.0
Epoch: 1300	Test loss: 0.0
Epoch: 1400	Test loss: 0.0
Epoch: 1500	Test loss: 0.0
Epoch: 1600	Test loss: 0.0
Epoch: 1700	Test loss: 0.0
Epoch: 1800	Test loss: 0.0
Epoch: 1900	Test loss: 0.0
Epoch: 2000	Test loss: 0.0
Epoch: 2100	Test loss: 0.0
Epoch: 2200	Test loss: 0.0
Epoch: 2300	Test loss: 0.0
Epoch: 2400	Test loss: 0.0


# Save

In [42]:
dati = actual_runs

In [26]:
def config2df(dati):
    df_data = pd.DataFrame(columns=['range_p', 'Num_nodes', 'Num_grafi_totali', 'learning_rate', 'batch_size', 'neurons_per_layer', 'correlations', 'error', 'test_loss', 'distance_of_means', 'batch_norm'])
    df_data['range_p'] = [d['graph_dataset']['range_p'] for d in dati]
    df_data['Num_nodes'] = [d['graph_dataset']['Num_nodes'] for d in dati]
    df_data['Num_grafi_totali'] = [d['graph_dataset']['Num_grafi_totali'] for d in dati]
    df_data['learning_rate'] = [d['training']['learning_rate'] for d in dati]
    df_data['batch_size'] = [d['training']['batch_size'] for d in dati]
    df_data['neurons_per_layer'] = [d['model']['neurons_per_layer'] for d in dati]
    df_data['correlations'] = [d.get('correlations') for d in dati]
    df_data['error'] = [d.get('error') for d in dati]
    df_data['test_loss'] = [[round(l, 10) for l in d.get('test_loss') or []] for d in dati ]  # or [] : per il caso in cui d.get restituisca None
    df_data['distance_of_means'] = [d.get('distance_of_means') for d in dati]  # caso della classification
    #df_data['run_num'] = [d.get('run_num') for d in dati]
    df_data['batch_norm'] = [d['model']['put_batchnorm'] for d in dati]
    df_data['num_epochs'] = [d['training']['epochs'] for d in dati]
    return df_data

In [15]:
# correggo il dataframe esistente aggiungendo i campi che mi viene in mente di aggiungere nel tempo
file = "df_configs_class.csv" #"df_configs.csv" #
df_data = pd.read_csv(file)  
df_data['num_epochs'] = 5000
df_data.to_csv(file) 

In [43]:
df_data = config2df(dati)

In [45]:
df_data

,range_p,Num_nodes,Num_grafi_totali,learning_rate,batch_size,neurons_per_layer,correlations,error,test_loss,distance_of_means,batch_norm,num_epochs
0,"[0.05, 0.6]",260,1000,0.01,100,"[1, 32, 32]",None,None,"[0.0078486049, 0.0077183223, 0.0077000621, 0.0...",15.375359,True,2500


In [29]:
import os
outfile = "df_configs_class.csv" #"df_configs.csv" #
df_data.to_csv(outfile, index=False, mode='a', header=not os.path.exists(outfile))

In [30]:
# aggiungo al master config
configs.extend(actual_runs)

In [33]:
for c in configs:
    if c.get('test_loss'):
        del c['test_loss']

In [34]:
with open('configs_21-6-22.pickle', 'wb') as handle:
    pickle.dump(configs, handle, protocol=pickle.HIGHEST_PROTOCOL)